# Huffman Coding

In [1]:
import os
import heapq

class Binary_Tree_Node:
    
    def __init__(self,key,freq):
        # key is character
        # freq is number of freq character occurs
        self.key = key
        self.freq = freq
        self.left = None
        self.right = None
        
    ## comparing the two binary tree nodes with 2 overload function . to make easy to understand which binary node is lesser.
    def __lt__(self,other): 
        return self.freq < other.freq
    
    def __eq__(self,other):
        return self.freq == other.freq
        
class Huffman_Coding:
    def __init__(self,path):
        self.path = path
        self.__heap = []
        self.__codes = {} 
        self.__reverse_Code = {}
        
    def __build_heap(self,freq_dict):
        for key in freq_dict:
            freq = freq_dict[key].freq
            binary_tree_node = Binary_Tree_Node(key,freq)
            heapq.heappush(self.__heap,binary_tree_node)
        
    def __make_frequnecy_dict(self,text):
        freq_dict = {}
        for i in text:
            if i in freq_dict:
                freq_dict[i].freq += 1
            else:
                freq_dict[i] = Binary_Tree_Node(i, 1)
        return freq_dict  
    
    def __build_tree(self):
        
        while(len(self.__heap)>1):
            binarytree_min_node_1 = heapq.heappop(self.__heap)
            binarytree_min_node_2 = heapq.heappop(self.__heap)
            
            sum_freq = binarytree_min_node_1.freq + binarytree_min_node_2.freq
            new_node = Binary_Tree_Node(None,sum_freq)
            
            new_node.left = binarytree_min_node_1
            new_node.right = binarytree_min_node_2
            
            heapq.heappush(self.__heap,new_node)
        return 
    
    def __build_code_helper(self,root,curr_bits):
        
        if root is None:
            return
        
        if root.key is not None:
            self.__codes[root.key] = curr_bits
            self.__reverse_Code[curr_bits] = root.key
            return
        self.__build_code_helper(root.left,curr_bits+"0")
        self.__build_code_helper(root.right,curr_bits+"1")
    
    def __build_code(self):
        
        root = heapq.heappop(self.__heap)
        self.__build_code_helper(root,"")
        
    def __get_encoded_text(self,text):
        encoded_text = ""
        
        for char in text:
            encoded_text += self.__codes[char]
        return encoded_text
    
    def __padding_encoding_text(self,encoded_text):
        
        padded_amount = 8 - (len(encoded_text)%8)
        
        for i in range(padded_amount):
            encoded_text += "0"
        
        padded_info = "{0:08b}".format(padded_amount)    
        padded_encoded_text = padded_info + encoded_text
        
        return padded_encoded_text
    
    def __get_byte_array(self,padded_encoded_text):
        
        array = []
        
        for i in range(0,len(padded_encoded_text),8):
            byte = padded_encoded_text[i:i+8]
            array.append(int(byte,2))
            
        return array
    
    def compress(self):
        ## take file from path
        ## make text from the file
        file_name,file_extension = os.path.splitext(self.path)
        output_path = file_name + "_compressed.bin"
        
        with open(self.path,"r+") as file , open (output_path,"wb") as output:
            ## make frequency dictionary from the text
            text = file.read()
            text = text.rstrip()
            freq_dict = self.__make_frequnecy_dict(text)

            ## contruct the heap from the frequency dictionary
            self.__build_heap(freq_dict)

            ## contruct the binary tree from the heap
            self.__build_tree()

            ## contruct the code from the tree
            self.__build_code()

            ## contruct the encoded text from the code
            encoded_text = self.__get_encoded_text(text)

            ## padding the encoding text ( adding the extra code so that the encoding text is mulitple of 8)
            padded_encoded_text = self.__padding_encoding_text(encoded_text)

            ## transform the encoded text into binary file
            byte_array = self.__get_byte_array(padded_encoded_text)

            ## return the binary file as the output
            final_bytes = bytes(byte_array)
            output.write(final_bytes)
        print("Compressed")
        return output_path
    
    def __removing_the_padded_from_text(self,text):
        
        padded_information = text[0:8] ##starting of the 8 bits are the infor about the extra padd we added at the last of the bit_string
        extra_padded = int(padded_information,2) # number of padded (00000101 = 5)
        
        text = text[8:] ## here we remove the padded_infor bits from the text
        
        ## actual text is
        text_after_padding_removed = text[:-1*extra_padded] ## if extra_padded is 3  , so last 3 bits will be removed from the text
        return text_after_padding_removed
    
    def __decoded_text(self,text):
        decoded_text = ""
        current_bits = ""
        
        for bit in text:
            current_bits += bit
            if current_bits in self.__reverse_Code:
                character = self.__reverse_Code[current_bits]
                decoded_text += character
                current_bits = ""
        return decoded_text
                
        
    def decompress(self,input_path):
        
        filename, file_extension = os.path.splitext(self.path)
        output_path = filename + "_decompressed"+".txt"
        
        with open(input_path,'rb') as file, open(output_path,'w') as output:
            
            bit_string = ""
            
            byte = file.read(1)   ## byte = compressed_character like 0xfa
            
            while byte: ## till the bytes are there loop will run
                
                byte = ord(byte)## convert byte into integer number , 0xfa = 5

                bits = bin(byte)[2:].rjust(8,'0') ## converting integer (5) into binary bits like 0b101 and
                                                  ## taking the bits value like 101 and using padding adding "0" to fill it
                                                  ## so that the bit becomes multiple of 8
                        
                bit_string += bits## storing bits in bit_string
        
                byte = file.read(1) ## again reading the byte from the file , like 0xfb
                
            ## remove the extra padded bits and get the actual text bit
            actual_text = self.__removing_the_padded_from_text(bit_string)
            
            ## decode the bit into the text(character)
            decompressed_text = self.__decoded_text(actual_text)
            output.write(decompressed_text)
            print("Decompressed")
        return
            
path ="D:\PYTHON\Coding Ninja\Dataset\Sample.txt"
h = Huffman_Coding(path)
output_path = h.compress()
h.decompress(output_path)

Compressed
Decompressed
